In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import networkx as nx
import random

In [ ]:

def cascading_failure(G,H,thetha):
################################ Cascading failure ######################################
    initial_node = random.choice([i for i in list(H.nodes)])
    initial_node = random.sample([i for i in list(H.nodes)],12)

    H.remove_nodes_from(initial_node)         #randomly removing a node, avoiding the immume_nodes, to initiate the cascade
    not_failed_nodes =[]
    failed_nodes = []
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        for each in current_failed_nodes:
            H.remove_node(each)
            if each not in failed_nodes:
                failed_nodes.append(each)

        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}
        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha:
                count +=1
        if count == len(not_failed_nodes):
            break
         
    return failed_nodes



def G(q_0,average_degree,degrees_poisson,threshold):
    sum_k = 0
    
    for k in range(1, max(degrees_poisson)+1):
        sum_m = 0
        for m in range(0, k):
            if threshold < (m/k):
#                 print("if")
                F = 1
            else:
                F = 0
            sum_m += (math.factorial(k-1) / (math.factorial(m)*math.factorial(k-1-m)))*pow(q_0,m)*pow((1-q_0), k-1-m)*F
        sum_k += (k/average_degree)*(list(degrees_poisson).count(k)/num_nodes)*sum_m
        
    q_1 = rho_0 + (1-rho_0)*sum_k  #q_1

    if abs(q_1-q_0) < 0.00001:
        return q_1
    else:
        return G(q_1,average_degree,degrees_poisson,threshold)

def Rho(q_inf,degrees_poisson,threshold,rho_0):
    sum_K = 0
    for k in range(1, max(degrees_poisson)+1):
        sum_M = 0
        for m in range(0, k+1):
            if threshold < (m/k):
                F = 1
            else:
                F = 0
            sum_M += (math.factorial(k) / (math.factorial(m)*math.factorial(k-m)))*pow(q_inf,m)*pow((1-q_inf), k-m)*F
        sum_K +=(list(degrees_poisson).count(k)/num_nodes)*sum_M

    rho = rho_0 + (1-rho_0)*sum_K  #q_1
    return rho

################################### Analytic result ###################################

num_nodes = 30000
rho_0 = 0.0006 #initially 16 active nodes 
avg_degree = 8
realizations = 1
thresholds = np.arange(0.075,0.25, 0.0125)
# threshold_simulation = [0.85,  0.855, 0.86,  0.865, 0.87,  0.875, 0.88 , 0.885, 0.89 , 0.895 ,0.9]

# thresholds = [1-i for i in threshold_simulation[::-1]]
suriviving_prob_array_analytic = []

for i in thresholds:
    intermediate_suriviving_prob_array_analytic = []
    for j in range(realizations):
        degrees_poisson = np.random.poisson(avg_degree, num_nodes)
        average_degree = np.mean(degrees_poisson)
        intermediate_suriviving_prob_array_analytic.append(Rho(G(rho_0,average_degree,degrees_poisson,i),degrees_poisson, i,rho_0))
    suriviving_prob_array_analytic.append(round(np.mean(intermediate_suriviving_prob_array_analytic),3))

##################################### Simulation result ##################################   

threshold_simulation = [1-i for i in thresholds[::-1]]

N_nodes = 30000
# p = 0.008
m = 120000
average_degree_of_original_graph = (2*m) /N_nodes 


realizations = 1
suriviving_prob_array = []
for thetha in threshold_simulation:
    intermediate_suriviving_prob_array = []
    for i in range(realizations):
        G = nx.gnm_random_graph(N_nodes,m)
        degree_sequence_erdos = list(G.degree())
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])
        remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
        G.remove_nodes_from(remove)
        N_nodes_new = list(G.nodes)    #list of nodes with k>0
        H = G.copy()   # graph contains nodes that have k > 0
        I = G.copy()
        degree_sequence_erdos = list(H.degree())
        failed_nodes_at_the_end_cascade = cascading_failure(G,H,thetha)
        suriviving_prob = (len(I.nodes) - len(failed_nodes_at_the_end_cascade))/ len(I.nodes)
        intermediate_suriviving_prob_array.append(suriviving_prob)
    suriviving_prob_array.append(round(np.mean(intermediate_suriviving_prob_array),2))
   


thresholds_adjusted = [1-i for i in thresholds[::-1]]


plt.scatter(thresholds_adjusted, suriviving_prob_array, marker='o', facecolors='none', edgecolors='red', s=80, linewidths=1.5,label = "Simulation result")
plt.plot(thresholds_adjusted,suriviving_prob_array_analytic,'o', label = "Analytic result" )
print(suriviving_prob_array)
print(suriviving_prob_array_analytic)
print(thresholds_adjusted)
# print(adjusted_threshold)
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.show()

        